In [58]:
import itertools
import numpy as np
import pandas as pd
import sys
import os
from datetime import timedelta

print(os.getcwd())

/Users/alexander.prams/hodschallenge


In [59]:
in_file = "challenge/data/device_activations.csv"

In [149]:
previous_readings = pd.read_csv(in_file)
print(type(previous_readings['time'][0]))
previous_readings['time'] = pd.to_datetime(previous_readings['time'])
device_list = sorted(previous_readings['device'].unique())
print(type(previous_readings['time'][0]))
print(previous_readings.loc[0, :]['time'])
print(previous_readings.loc[1, :]['time'])
print(previous_readings.loc[2, :]['time'])

<class 'str'>
['device_1', 'device_2', 'device_3', 'device_4', 'device_5', 'device_6', 'device_7']
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016-07-01 04:23:32
2016-07-01 06:52:57
2016-07-01 06:53:00


In [61]:
print("Number of data points: ", len(previous_readings))

Number of data points:  9045


In [105]:
def date_time_same_hour(dt1, dt2):
    same_until_hour = dt1.year == dt2.year and dt1.month == dt2.month and dt1.day == dt2.day and dt1.hour == dt2.hour
    if same_until_hour:
        return True
    else:
        return False


In [164]:
# IDEA: ADD HOUR & DAY AS FEATURE?
def preallocate_features(previous_readings):
    n_devices = previous_readings['device'].nunique()
    print('n_devices = ', n_devices)
    tmp_time_stamp = first_time_stamp = pd.to_datetime(previous_readings['time'][0])
    
    unique_hour_date_times = [first_time_stamp.replace(minute=0, second=0)]
    
    hour_interval_start_end = pd.date_range(first_time_stamp.replace(minute=0, second=0), 
                                            previous_readings['time'][len(previous_readings) - 1].replace(minute=0, second=0), 
                                           freq='H')
    print(hour_interval_start_end)
    
    for date_time in previous_readings['time']:
        if not date_time_same_hour(date_time, tmp_time_stamp):
            unique_hour_date_times += [date_time.replace(minute=0, second=0)]
        tmp_time_stamp = date_time
    
    
    n_unique_hours = len(unique_hour_date_times)
    data=np.zeros([n_unique_hours, n_devices, 1])
    features = pd.DataFrame(0, index=np.arange(hour_interval_start_end), 
                            columns=['time'] + device_list)
    features['time'] = hour_interval_start_end
    return features
#print("Unique hours: ", len(preallocate_features(previous_readings)))
print(preallocate_features(previous_readings))
    

n_devices =  7
DatetimeIndex(['2016-07-01 04:00:00', '2016-07-01 05:00:00',
               '2016-07-01 06:00:00', '2016-07-01 07:00:00',
               '2016-07-01 08:00:00', '2016-07-01 09:00:00',
               '2016-07-01 10:00:00', '2016-07-01 11:00:00',
               '2016-07-01 12:00:00', '2016-07-01 13:00:00',
               ...
               '2016-08-31 08:00:00', '2016-08-31 09:00:00',
               '2016-08-31 10:00:00', '2016-08-31 11:00:00',
               '2016-08-31 12:00:00', '2016-08-31 13:00:00',
               '2016-08-31 14:00:00', '2016-08-31 15:00:00',
               '2016-08-31 16:00:00', '2016-08-31 17:00:00'],
              dtype='datetime64[ns]', length=1478, freq='H')


TypeError: invalid type promotion

In [160]:
def preprocess_features(previous_readings):
    """
    Generate features so that we have an array with dimensions [T, n_devices, 1]
    T: time
    n_devices: number of devices
    Last dimension: number of activations per hour
    """
    
    # We preallocate to avoid many appends (append copies according to pandas docs, might become an issue/slow for large data)
    features = preallocate_features(previous_readings)
    cur_feature_index = 0
    cur_feature_dt = features['time'][0]
    for index, row in previous_readings.iterrows():
        if not(row['time'] > cur_feature_dt and row['time'] < cur_feature_dt + timedelta(hours=1)):
            cur_feature_index += 1
            cur_feature_dt = features['time'][cur_feature_index]
        # Increment device's counter at time
        features.at[cur_feature_index, row['device']] += 1
            
        features['time']
    print(features)
    return 

preprocess_features(previous_readings)

n_devices =  7
(541, 7, 1)
                   time  device_1  device_2  device_3  device_4  device_5  \
0   2016-07-01 04:00:00         0         0         0         0         0   
1   2016-07-01 06:00:00         0         3         0         0         0   
2   2016-07-01 07:00:00         0        12         0         4         0   
3   2016-07-01 08:00:00         0         6         2         3         3   
4   2016-07-01 09:00:00         0         4         1         0         0   
5   2016-07-01 10:00:00         0         4         5         0         0   
6   2016-07-01 11:00:00         0         5         1         0         0   
7   2016-07-01 12:00:00         0         8         4         0         0   
8   2016-07-01 13:00:00         0         7         5         0         0   
9   2016-07-01 14:00:00         0         4         3         0         0   
10  2016-07-01 15:00:00         0         9         4         0         0   
11  2016-07-01 16:00:00         0         7      